# Notebook to evaluate characteristics of individual anomaly trajectories


Main difference to evaluation in `evaluate_scores.iypnb`: there we performed a population view taking into account all samples that we have, here it is an individual trajectory perspective. This individual trajectory perspective yields a much smaller sample size due to the restriction that each analyzed trajectory must have a microbial sample in 3 months preceding abx exposure (at -1.0) as well as at least one sample after exposure:

| n | 1 |2 |
|----------|----------|----------|
| # infants     | 49 |30 |
| # infants w score increased + w/o | 21 + 28 |11 + 19 |

As contrast, sample size in population view performed in `evaluate_scores.iypnb`:

| Characteristic | n = 1 | n = 2 |
|-----------------|--------|--------|
| **ABX Duration** |        |        |
| < 7 days        | 39     | 31     |
| ≥ 7 days        | 51     | 53     |
| **Age**         |        |        |
| < 12 months     | 75     | 35     |
| 12-24 months    | 34     | 38     |

To run this notebook you need to create and activate the following conda environment:

```
conda create --name score_eval -c conda-forge -c defaults numpy pandas matplotlib seaborn scipy scikit-learn ipython ipykernel -y
conda activate score_eval
pip install -e .
```

## Setup

In [ ]:
import os
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu

from src.utils_eval_score import (
    _plot_score_after_nth_abx_exposure,
    _select_samples_around_nth_abx_exposure,
    get_scores_n_abx_info,
    plot_trajectory,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams.update({"font.family": "DejaVu Sans"})
plt.style.use("tableau-colorblind10")

warnings.filterwarnings("ignore", category=FutureWarning)
# avg. number of days per month
DAYS_PER_MONTH = 30.437

USER input: define the inferred model and linked datasets to evaluate here:

In [ ]:
#### USER INPUT START
# name of the model
model_name = "saved_models_microbial_novel_alpha_div2/id-55"
# which model version to evaluate: "best" or "last"
point_to_evaluate = "best"

# name of feature dataset used for model
ft_name = "ft_vat19_anomaly_v20240806_entero_genus"
# name of abx time-series used for model
abx_ts_name = "ts_vat19_abx_v20240806"

# limit evaluation to time range up to this many months (if None no limit is set
# and all scores are evaluated)
limit_months = 24.0

# whether to group samples prior to abx exposure in analysis
group_samples = True

# ! which abx exposure to analyse in this notebook
n = 1

# ! how many samples prior and after abx exposure to consider
min_samples = -3.0
# 3.0 for n=1, 4.0 for n=2
max_samples = 3.0  

# whether to filter noabx score samples by having at least 1 obs prior to cutoff
no_filter = True

# whether to display diet info after 1st, 2nd, 3rd abx exposure plots
display_diet_info = True

# whether to have max. resolution of 0.5 months or not
max_resolution = False

# scaling factor options:
scaling_factors_used = True

# if scaling_factors_used is True, then the following options are required:
# non-centered = "nc_std" or centered = "std"
stddev_type = "nc_std"
# moving average window size: 30 or 10
moving_avg = 10
# whether to include duplicates: "--RD-True" or ""
duplicates = "--RD-False"
# using lower bound of 1 for SFs: "lower_bound-1" or ""
lower_bound = ""

#### USER INPUT END

## Prepare data

In [ ]:
base_path = f"../data/{model_name}/anomaly_detection/"

res_n_group = f"g{str(group_samples)[0]}_maxres{str(max_resolution)[0]}"

if scaling_factors_used:
    print("Scaling factors used.")
    folder_name = f"using-SF_{stddev_type}_z_scores--moving_avg-{moving_avg}-cummax{lower_bound}{duplicates}"

    scores_path = f"{base_path}scores_{point_to_evaluate}_normal/{folder_name}/"
    evaluation_path = f"{base_path}evaluation_{point_to_evaluate}_overall_{res_n_group}_{stddev_type}_ma{moving_avg}{duplicates.replace('-', '_').lower()}/"
else:
    scores_path = f"{base_path}scores_{point_to_evaluate}_normal/"
    evaluation_path = (
        f"{base_path}evaluation_{point_to_evaluate}_overall_{res_n_group}_no_scaling/"
    )
if not os.path.exists(evaluation_path):
    os.makedirs(evaluation_path)


In [ ]:
noabx_train, noabx_val, abx_scores_flat, abx_df, abx_age_at_all = get_scores_n_abx_info(
    scores_path, ft_name, limit_months, abx_ts_name, no_filter=no_filter
)


In [ ]:
abx_events = pd.DataFrame()
abx_events["host_id"] = abx_scores_flat["host_id"].unique()
abx_events = pd.merge(abx_events, abx_df, on="host_id", how="left")

## Trajectories with large vs. no increase after abx

In [ ]:
# SELECT & PREPARE SAMPLES
score_col = f"score_{n}"

scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
    abx_scores_flat,
    abx_df,
    n=n,
    min_samples=min_samples,
    max_samples=max_samples,
    group_samples=group_samples,
    score_var=score_col,
    max_resolution=max_resolution,
)

scores_abx_nth_samples.drop(
    columns=[
        "abx",
        "score_0",
        "div_alpha_observed_features",
        "div_alpha_faith_pd",
        "div_alpha_shannon",
        "max_abx_w_microbiome",
        "abx_max_count_ever",
    ],
    inplace=True,
)

# select only samples that have a -1 score
hosts_w_prior_sample = scores_abx_nth_samples[
    scores_abx_nth_samples["diff_age_nth_abx"] == -1.0
].host_id.unique()
len(hosts_w_prior_sample)

print(scores_abx_nth_samples.shape)
scores_abx_nth_samples = scores_abx_nth_samples[
    scores_abx_nth_samples.host_id.isin(hosts_w_prior_sample)
]
print(scores_abx_nth_samples.shape)

In [ ]:
# Compute relative score: max score in 3 or 4 months after abx exposure
scores_abx_nth_samples = (
    scores_abx_nth_samples.groupby("host_id")
    .apply(
        lambda x: x.assign(
            score_rel_change_1=x[score_col] / x[score_col].shift(1),
            score_rel_change_2=x[score_col] / x[score_col].shift(2),
            score_rel_change_3=x[score_col] / x[score_col].shift(3),
            score_rel_change_4=x[score_col] / x[score_col].shift(4),
        ),
        include_groups=False,
    )
    .reset_index()
)
scores_abx_nth_samples.drop(columns="level_1", inplace=True)

if n == 1:
    scores_abx_nth_samples.drop(columns="score_rel_change_4", inplace=True)

# Identify all columns relevant to score_rel_change
rel_change_cols = [
    col for col in scores_abx_nth_samples.columns if col.startswith("score_rel_change_")
]
# Select those columns, forward-fill each row, and pick the last non-NaN entry
scores_abx_nth_samples["score_rel_change"] = (
    scores_abx_nth_samples[rel_change_cols]
    .ffill(axis=1)  # fill forward along each row
    .iloc[:, -1]  # take the rightmost (last) non-NaN value
)
df_max_rel_change = (
    scores_abx_nth_samples.groupby("host_id", as_index=False)["score_rel_change"]
    .max()
    .rename(columns={"score_rel_change": "max_score_rel_change"})
)
df_max_rel_change.dropna(inplace=True)

print(
    f"Number of unique infants with quantifiable rel score change: {df_max_rel_change.host_id.nunique()}"
)

In [ ]:
scores_abx_nth_samples["months_since_last_prior_sample"] = (
    scores_abx_nth_samples["age_nth_abx"] - scores_abx_nth_samples["month5_bin"]
)

scores_abx_nth_samples[
    [
        "host_id",
        "diff_age_nth_abx",
        "age_nth_abx",
        "month5_bin",
        "months_since_last_prior_sample",
    ]
]

In [ ]:
scores_abx_nth_samples["gastroenteritis"] = "Others"
scores_abx_nth_samples.loc[
    scores_abx_nth_samples["abx_reason"] == "Gastroenteritis", "gastroenteritis"
] = "Gastroenteritis"

scores_abx_nth_samples["gastroenteritis"].value_counts(dropna=False)

In [ ]:
# get abx duration for these infants
df_duration = scores_abx_nth_samples.groupby("host_id", as_index=False)[
    "abx_any_last_dur_days"
].last()
df_duration.dropna(inplace=True)

df_rel_change = pd.merge(df_max_rel_change, df_duration, on="host_id", how="left")
print(
    f"Number of unique infants with quantifiable rel score change + abx duration: {df_rel_change.host_id.nunique()}"
)

# get all other characteristics in -1 bin before abx exposure
characteristics_numeric = ["month5_bin", f"score_{n}", "months_since_last_prior_sample"]

characteristics_cat = [
    "diet_milk",
    "diet_weaning",
    "delivery_mode",
    "abx_reason",
    "gastroenteritis",
    "abx_type",
]

cols_to_select = ["host_id"] + characteristics_numeric + characteristics_cat
df_charc = scores_abx_nth_samples.loc[
    scores_abx_nth_samples["diff_age_nth_abx"] == -1.0, cols_to_select
]

df_rel_change_final = pd.merge(df_rel_change, df_charc, on="host_id", how="left")
print(
    f"Number of unique infants with quantifiable rel score change + abx duration + chars: {df_rel_change_final.host_id.nunique()}"
)

# transform categoricals to proper categoricals:
for cat in characteristics_cat:
    df_rel_change_final[cat] = df_rel_change_final[cat].astype("category")

df_rel_change_final.head()

In [ ]:
# DEFINE GROUPS
# thresh = df_rel_change_final["max_score_rel_change"].quantile(0.75)
# print(thresh)
thresh = 2.0

rel_increase = df_rel_change_final.loc[
    df_rel_change_final["max_score_rel_change"] >= thresh
]
print(rel_increase.host_id.nunique())

rel_no_increase = df_rel_change_final.loc[
    df_rel_change_final["max_score_rel_change"] < thresh
]
print(rel_no_increase.host_id.nunique())

In [ ]:
characteristics_numeric += ["abx_any_last_dur_days"]

In [ ]:
# Describe CHARACTERISTICS
results_df = pd.DataFrame()

# Aggregate results for each group
for label, group in [
    ("increase", rel_increase),
    ("no_increase", rel_no_increase),
]:
    # nb infants
    nb_infants = group.host_id.nunique()
    results_df.loc[label, "nb_infants"] = nb_infants

    # Summaries for numeric columns
    for c in characteristics_numeric:
        stats = group[c].describe()
        mean_val = round(stats["mean"], 2)
        std_val = round(stats["std"], 2)
        results_df.loc[label, c] = f"{mean_val} ± {std_val}"

    # Fractions for each category value
    for cat in characteristics_cat:
        cat_counts = group[cat].value_counts(dropna=False)
        for val, cnt in cat_counts.items():
            fraction = round(cnt / nb_infants, 2)
            results_df.loc[label, f"{cat}_{val}"] = fraction
display(results_df)


In [ ]:
# STATISTICAL TESTS for characteristics
sign_numeric_df = pd.DataFrame()

# Mann-Whitney U for numeric columns
for c in characteristics_numeric:
    inc_vals = rel_increase[c].dropna()
    no_inc_vals = rel_no_increase[c].dropna()
    stat, p_val = mannwhitneyu(inc_vals, no_inc_vals, alternative="two-sided")
    sign_numeric_df.loc[c, "statistic"] = round(stat, 4)
    sign_numeric_df.loc[c, "p-value"] = p_val

# Chi-square for categorical columns
sign_cat_df = pd.DataFrame()
for cat in characteristics_cat:
    inc_c = rel_increase[cat].dropna().value_counts()
    no_c = rel_no_increase[cat].dropna().value_counts()
    contingency = pd.DataFrame([inc_c, no_c], index=["increase", "no_increase"]).fillna(
        0
    )
    chi2, p_val, dof, expected = chi2_contingency(contingency)
    sign_cat_df.loc[cat, "chi2_statistic"] = round(chi2, 4)
    sign_cat_df.loc[cat, "p-value"] = p_val

print("\nNumeric Significance Results:")
print(sign_numeric_df)
print("\nCategorical Significance Results:")
print(sign_cat_df)

In [ ]:
# ----------------------------
# CREATE SUBPLOTS
#   2 rows x 5 columns to accommodate 5 categorical variables (row 1)
#   and 2 numeric variables (row 2, columns 0 & 1 only)
# ----------------------------
n_rows = len(characteristics_numeric) + len(characteristics_cat)
fig, axs = plt.subplots(
    nrows=n_rows, ncols=1, figsize=(8, 20), dpi=600
)  # , sharex=True)
# fig.suptitle("Comparison of Characteristics for 'increase' vs. 'no_increase'", fontsize=12, y=1.02)

for idx, c in enumerate(characteristics_numeric):
    increase_vals = rel_increase[c].dropna()
    no_increase_vals = rel_no_increase[c].dropna()

    axs[idx].boxplot(
        [increase_vals, no_increase_vals], labels=["increase", "no_increase"]
    )
    axs[idx].set_ylabel(c)

# Stacked barplots for each categorical variable
for idx_cat, cat in enumerate(characteristics_cat):
    idx_cat_true = idx + idx_cat + 1
    cat_fraction_cols = [col for col in results_df.columns if col.startswith(cat + "_")]
    results_df[cat_fraction_cols].plot(
        kind="bar",
        stacked=True,
        ax=axs[idx_cat_true],
    )
    axs[idx_cat_true].set_ylabel(
        f"{cat} fractions",
    )
    # Remove the prefix for cleaner legend labels
    handles, labels = axs[idx_cat_true].get_legend_handles_labels()
    new_labels = [label.replace(cat + "_", "") for label in labels]
    axs[idx_cat_true].legend(
        handles, new_labels, title=cat, bbox_to_anchor=(1.05, 1), loc="upper left"
    )

# Remove x-axis ticks and labels from all subplots except the last one
for ax in axs[:-1]:
    ax.set_xlabel("")
    ax.set_xticklabels([])
for tick in axs[-1].get_xticklabels():
    tick.set_rotation(0)
plt.tight_layout()
plt.show()


In [ ]:
group_increase_scores = scores_abx_nth_samples.loc[
    scores_abx_nth_samples["host_id"].isin(rel_increase["host_id"])
]
_plot_score_after_nth_abx_exposure(
    group_increase_scores,
    x_axis="diff_age_nth_abx",
    y_axis=score_col,
    n=n,
    path_to_save=None,
    flag=score_col,
    tag=": relative score increase",
    min_samples=min_samples,
    max_samples=max_samples,
    max_resolution=max_resolution,
    grouped_samples=group_samples,
)

group_no_increase_scores = scores_abx_nth_samples.loc[
    scores_abx_nth_samples["host_id"].isin(rel_no_increase["host_id"])
]
_plot_score_after_nth_abx_exposure(
    group_no_increase_scores,
    x_axis="diff_age_nth_abx",
    y_axis=score_col,
    n=n,
    path_to_save=None,
    flag=score_col,
    tag=": relative score no increase",
    min_samples=min_samples,
    max_samples=max_samples,
    max_resolution=max_resolution,
    grouped_samples=group_samples,
)

In [ ]:
# plot individual trajectories for infants with increased scores prior to abx exposure:

infants_w_no_increase = rel_no_increase["host_id"].unique().tolist()

for infant in infants_w_no_increase:
    plot_trajectory(
        abx_scores_flat,
        abx_events,
        infant,
        [f"score_{n}"],
        path_to_output=None,
        flag=f"score_{n}",
    )